In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

ontology_url = 'gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/tanks_ontology.csv'

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
df = pd.read_csv(ontology_url)

In [4]:
df.isna().sum()

subject        0
predicate      0
object       445
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df

,subject,predicate,object
0,player_2685498_stats_on_15697,concreteTankPiercings,72
1,player_4991449_stats_on_3681,concreteTankAvgDamageBlocked,686.84
2,player_12177534_stats_on_19969,concreteTankPiercings,1529
3,player_12050543_stats_on_2433,concreteTankDamageReceived,138189
4,player_14636191_stats_on_15697,concreteTankDirectHitsReceived,176
...,...,...,...
4595023,player_6106224_stats_on_13825,concreteTankLosses,251
4595024,player_20304852_stats_on_13089,concreteTankBattleAvgXp,499
4595025,player_3202392_stats_on_63537,concreteTankFrags,78
4595026,player_4865908_stats_on_2433,concreteTankMarkOfMastery,4


In [ ]:
!pip install git+https://github.com/pykeen/pykeen.git

In [ ]:
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

tf = TriplesFactory.from_labeled_triples(
  df[["subject", "predicate", "object"]].values
)

training, testing, validation = tf.split([.8, .1, .1])

In [ ]:
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='RotatE',
    stopper='early',
    epochs=20,
    dimensions=512,
    random_seed=420
)

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /root/.data/pykeen/checkpoints/best-model-weights-5ddf0ec2-9369-4321-b7d1-f74744ecfea4.pt


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/14359 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/14359 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/14359 [00:00<?, ?batch/s]

In [ ]:
from pykeen.models.predict import get_tail_prediction_df

df = get_tail_prediction_df(result.model, 'player_IVIargo', 'clan_101221', triples_factory=result.training)
print(df.head(5))

# весь список метрик доступен через to_flat_dict()
print(f"MRR: {result.metric_results.to_flat_dict()['both.realistic.inverse_harmonic_mean_rank']}")
for k in [1,3,5,10]:
    print(f"Hits@{k} : {result.metric_results.to_flat_dict()['both.realistic.hits_at_'+str(k)]}")